## 30s k-fold

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, AveragePooling1D, Dropout
from keras.layers import Activation, BatchNormalization
from keras.optimizers import Adam
from keras.utils import np_utils
import tensorflow as tf
from keras.callbacks import TensorBoard, EarlyStopping
np.set_printoptions(suppress=True)

trainD = np.load("/home/hsiehch/30s/train_data.npy")
trainL = np.load("/home/hsiehch/30s/train_label.npy")
validationD = np.load("/home/hsiehch/30s/validation_data.npy")
validationL = np.load("/home/hsiehch/30s/validation_label.npy")
testD = np.load("/home/hsiehch/30s/test_data.npy")
testL = np.load("/home/hsiehch/30s/test_label.npy")

trainD = np.append(trainD, validationD, axis=0)
trainL = np.append(trainL, validationL, axis=0)
trainD = np.append(trainD, testD, axis=0)
trainL = np.append(trainL, testL, axis=0)

trainData = trainD.reshape((trainD.shape[0], trainD.shape[1], 1))
trainLabel = np_utils.to_categorical(trainL, 4)
print('Train Data:', trainData.shape)
print('Train Label: ', trainLabel.shape)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train Data: (10151, 9000, 1)
Train Label:  (10151, 4)


In [2]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True)
print(kf)

training_data = []
training_label = []
validation_cate_label = []
validation_data = []
validation_label = []

for train_index, test_index in kf.split(trainData):
    print('trian:', train_index, 'len', len(train_index), 'test:', test_index, 'len', len(test_index))
    training_data.append(trainData[train_index])
    training_label.append(trainLabel[train_index])
    validation_data.append(trainData[test_index])
    validation_label.append(trainLabel[test_index])
    validation_cate_label.append(trainL[test_index])

training_data = np.array(training_data)
training_label = np.array(training_label)
validation_data = np.array(validation_data)
validation_label = np.array(validation_label)
validation_cate_label = np.array(validation_cate_label)


KFold(n_splits=5, random_state=None, shuffle=True)
trian: [    0     1     2 ... 10148 10149 10150] len 8120 test: [    3     7    13 ... 10127 10132 10140] len 2031
trian: [    0     1     2 ... 10148 10149 10150] len 8121 test: [    6    10    11 ... 10141 10145 10146] len 2030
trian: [    0     2     3 ... 10148 10149 10150] len 8121 test: [    1     5     8 ... 10134 10142 10147] len 2030
trian: [    1     3     4 ... 10146 10147 10148] len 8121 test: [    0     2    14 ... 10135 10149 10150] len 2030
trian: [    0     1     2 ... 10147 10149 10150] len 8121 test: [    4    18    26 ... 10143 10144 10148] len 2030


In [3]:
training_data[0].shape

(8120, 9000, 1)

In [4]:
from sklearn.metrics import f1_score
import pandas as pd
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
import itertools

def create_model():
    model = Sequential() 
    model.add(Conv1D(filters = 32, kernel_size = 7, input_shape = (trainData.shape[1], 1)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 32, kernel_size = 7))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 64, kernel_size = 7))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 64, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 128, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 128, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Dropout(0.5))

    model.add(Conv1D(filters = 256, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))

    model.add(Conv1D(filters = 256, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Dropout(0.5))

    model.add(Conv1D(filters = 512, kernel_size = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Dropout(0.5))

    model.add(Conv1D(filters = 512, kernel_size = 3))
    model.add(Activation('relu'))

    model.add(Flatten())
    model.add(Dense(128, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation = 'relu'))
    model.add(Dense(4, activation = "softmax"))

    return model

def calculate_avg_F1(scores):
    tmp = [0,0,0,0]
    for i in scores:
        tmp += i
    tmp /= 5
    return tmp

def run_model(batch_S, learning_rate):
    
    Scores = []
    for X_train, Y_train, X_val, Y_val, val_cat in zip(training_data, training_label, validation_data, validation_label, validation_cate_label):

        model = create_model()
        adam = Adam(lr = learning_rate)
        model.compile(optimizer = adam, loss = "categorical_crossentropy", metrics=['accuracy'])
        early_stop = EarlyStopping(patience=20)
        train_history_1 = model.fit(x = X_train, 
                                    y = Y_train,
                                    epochs=80,
                                    validation_data=(X_val, Y_val),
                                    callbacks=[early_stop],
                                    batch_size=batch_S, 
                                    verbose=1)
        evaluation = model.evaluate(x = X_val, y = Y_val)
        print('Loss: {:.3f}, Accuracy: {:.3f}'.format(evaluation[0], evaluation[1]))

        validation_prediction = model.predict_classes(X_val, batch_size=batch_S)
        score = f1_score(val_cat, validation_prediction, average=None)
        Scores.append(score)
        del model
    
    result = calculate_avg_F1(Scores)
    F1_result.append(result)
    del Scores
    

        
F1_result = []
hyperparameter_index = []
batch_S = [30, 50, 70, 90, 120]
learning_rate = [0.00005, 0.0001, 0.0005, 0.001, 0.005]

for bat, lr in itertools.product(batch_S, learning_rate):
    hyperparameter_index.append([bat, lr])
    run_model(bat, lr)

print('Finish training!')

Train on 8120 samples, validate on 2031 samples
Epoch 1/80
8120/8120 [==============================] - 16s 2ms/step - loss: 1.0560 - acc: 0.5698 - val_loss: 1.1379 - val_acc: 0.5879
Epoch 2/80
8120/8120 [==============================] - 10s 1ms/step - loss: 0.9978 - acc: 0.5844 - val_loss: 1.0750 - val_acc: 0.5879
Epoch 3/80
8120/8120 [==============================] - 10s 1ms/step - loss: 0.9668 - acc: 0.5841 - val_loss: 0.9617 - val_acc: 0.5879
Epoch 4/80
8120/8120 [==============================] - 10s 1ms/step - loss: 0.9078 - acc: 0.6017 - val_loss: 0.8685 - val_acc: 0.6268
Epoch 5/80
8120/8120 [==============================] - 10s 1ms/step - loss: 0.8600 - acc: 0.6309 - val_loss: 0.8281 - val_acc: 0.6583
Epoch 6/80
8120/8120 [==============================] - 10s 1ms/step - loss: 0.8242 - acc: 0.6499 - val_loss: 0.8375 - val_acc: 0.6578
Epoch 7/80
8120/8120 [==============================] - 10s 1ms/step - loss: 0.7976 - acc: 0.6533 - val_loss: 0.7979 - val_acc: 0.6785
Epoch 8

Epoch 61/80
8120/8120 [==============================] - 10s 1ms/step - loss: 0.3733 - acc: 0.8640 - val_loss: 0.4716 - val_acc: 0.8149
Epoch 62/80
8120/8120 [==============================] - 10s 1ms/step - loss: 0.3690 - acc: 0.8618 - val_loss: 0.4680 - val_acc: 0.8262
Epoch 63/80
8120/8120 [==============================] - 10s 1ms/step - loss: 0.3646 - acc: 0.8716 - val_loss: 0.4508 - val_acc: 0.8232
Epoch 64/80
8120/8120 [==============================] - 10s 1ms/step - loss: 0.3717 - acc: 0.8599 - val_loss: 0.4469 - val_acc: 0.8252
Epoch 65/80
8120/8120 [==============================] - 10s 1ms/step - loss: 0.3596 - acc: 0.8693 - val_loss: 0.4467 - val_acc: 0.8321
Epoch 66/80
8120/8120 [==============================] - 10s 1ms/step - loss: 0.3542 - acc: 0.8695 - val_loss: 0.4539 - val_acc: 0.8306
Epoch 67/80
8120/8120 [==============================] - 10s 1ms/step - loss: 0.3505 - acc: 0.8703 - val_loss: 0.4629 - val_acc: 0.8247
Epoch 68/80
8120/8120 [=========================

8121/8121 [==============================] - 10s 1ms/step - loss: 0.6034 - acc: 0.7490 - val_loss: 0.5967 - val_acc: 0.7488
Epoch 20/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.5952 - acc: 0.7518 - val_loss: 0.5785 - val_acc: 0.7640
Epoch 21/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.5844 - acc: 0.7587 - val_loss: 0.5525 - val_acc: 0.7700
Epoch 22/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.5576 - acc: 0.7727 - val_loss: 0.5516 - val_acc: 0.7749
Epoch 23/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.5603 - acc: 0.7700 - val_loss: 0.5567 - val_acc: 0.7793
Epoch 24/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.5452 - acc: 0.7796 - val_loss: 0.5281 - val_acc: 0.7857
Epoch 25/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.5424 - acc: 0.7868 - val_loss: 0.5424 - val_acc: 0.7842
Epoch 26/80
8121/8121 [==============================] - 10s

8121/8121 [==============================] - 9s 1ms/step - loss: 0.3853 - acc: 0.8583 - val_loss: 0.4442 - val_acc: 0.8369
Epoch 59/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.3778 - acc: 0.8633 - val_loss: 0.4507 - val_acc: 0.8315
Epoch 60/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.3747 - acc: 0.8611 - val_loss: 0.4572 - val_acc: 0.8261
Epoch 61/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.3795 - acc: 0.8613 - val_loss: 0.4687 - val_acc: 0.8369
Epoch 62/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.3761 - acc: 0.8647 - val_loss: 0.4545 - val_acc: 0.8350
Epoch 63/80
8121/8121 [==============================] - 9s 1ms/step - loss: 0.3653 - acc: 0.8713 - val_loss: 0.4472 - val_acc: 0.8374
Epoch 64/80
8121/8121 [==============================] - 10s 1ms/step - loss: 0.3610 - acc: 0.8666 - val_loss: 0.4704 - val_acc: 0.8325
Epoch 65/80
8121/8121 [==============================] - 10s 1ms/

8120/8120 [==============================] - 10s 1ms/step - loss: 0.5143 - acc: 0.8034 - val_loss: 0.5493 - val_acc: 0.7774
Epoch 17/80
8120/8120 [==============================] - 10s 1ms/step - loss: 0.5050 - acc: 0.8083 - val_loss: 0.5057 - val_acc: 0.8070
Epoch 18/80
8120/8120 [==============================] - 9s 1ms/step - loss: 0.4909 - acc: 0.8150 - val_loss: 0.4781 - val_acc: 0.8198
Epoch 19/80
8120/8120 [==============================] - 10s 1ms/step - loss: 0.4742 - acc: 0.8241 - val_loss: 0.5016 - val_acc: 0.8149
Epoch 20/80
8120/8120 [==============================] - 10s 1ms/step - loss: 0.4751 - acc: 0.8246 - val_loss: 0.4886 - val_acc: 0.8134
Epoch 21/80
8120/8120 [==============================] - 10s 1ms/step - loss: 0.4637 - acc: 0.8270 - val_loss: 0.4725 - val_acc: 0.8144
Epoch 22/80
8120/8120 [==============================] - 10s 1ms/step - loss: 0.4530 - acc: 0.8296 - val_loss: 0.4654 - val_acc: 0.8267
Epoch 23/80
8120/8120 [==============================] - 9s 1

...

Epoch 37/80
8121/8121 [==============================] - 8s 945us/step - loss: 0.3383 - acc: 0.8751 - val_loss: 0.5005 - val_acc: 0.8256
Epoch 38/80
8121/8121 [==============================] - 8s 956us/step - loss: 0.3263 - acc: 0.8810 - val_loss: 0.4640 - val_acc: 0.8498
Epoch 39/80
8121/8121 [==============================] - 8s 946us/step - loss: 0.3240 - acc: 0.8828 - val_loss: 0.4478 - val_acc: 0.8493
Epoch 40/80
8121/8121 [==============================] - 8s 951us/step - loss: 0.3233 - acc: 0.8851 - val_loss: 0.4624 - val_acc: 0.8483
Epoch 41/80
8121/8121 [==============================] - 8s 994us/step - loss: 0.3064 - acc: 0.8873 - val_loss: 0.4578 - val_acc: 0.8483
Epoch 42/80
8121/8121 [==============================] - 8s 949us/step - loss: 0.3128 - acc: 0.8905 - val_loss: 0.4685 - val_acc: 0.8384
Epoch 43/80
2030/2030 [==============================] - 1s 481us/step
Loss: 0.470, Accuracy: 0.858
Train on 8121 samples, validate on 2030 samples
Epoch 1/80
8121/8121 [========

Epoch 9/80
8121/8121 [==============================] - 8s 959us/step - loss: 0.5486 - acc: 0.7876 - val_loss: 0.5155 - val_acc: 0.8212
Epoch 10/80
8121/8121 [==============================] - 8s 956us/step - loss: 0.5238 - acc: 0.8013 - val_loss: 0.5028 - val_acc: 0.8315
Epoch 11/80
8121/8121 [==============================] - 8s 945us/step - loss: 0.5330 - acc: 0.7972 - val_loss: 0.5856 - val_acc: 0.7833
Epoch 12/80
8121/8121 [==============================] - 8s 958us/step - loss: 0.5031 - acc: 0.8152 - val_loss: 0.5019 - val_acc: 0.8291
Epoch 13/80
8121/8121 [==============================] - 8s 958us/step - loss: 0.4906 - acc: 0.8196 - val_loss: 0.4875 - val_acc: 0.8330
Epoch 14/80
8121/8121 [==============================] - 8s 957us/step - loss: 0.4862 - acc: 0.8213 - val_loss: 0.5002 - val_acc: 0.8369
Epoch 15/80
8121/8121 [==============================] - 8s 951us/step - loss: 0.4619 - acc: 0.8303 - val_loss: 0.4849 - val_acc: 0.8414
Epoch 16/80
8121/8121 [===================

Epoch 19/80
8120/8120 [==============================] - 8s 954us/step - loss: 0.9942 - acc: 0.5868 - val_loss: 0.9891 - val_acc: 0.5879
Epoch 20/80
8120/8120 [==============================] - 8s 952us/step - loss: 0.9936 - acc: 0.5868 - val_loss: 0.9889 - val_acc: 0.5879
Epoch 21/80
8120/8120 [==============================] - 8s 957us/step - loss: 0.9933 - acc: 0.5868 - val_loss: 0.9904 - val_acc: 0.5879
Epoch 22/80
8120/8120 [==============================] - 8s 949us/step - loss: 0.9938 - acc: 0.5868 - val_loss: 0.9890 - val_acc: 0.5879
Epoch 23/80
8120/8120 [==============================] - 8s 950us/step - loss: 0.9935 - acc: 0.5868 - val_loss: 0.9890 - val_acc: 0.5879
Epoch 24/80
8120/8120 [==============================] - 8s 950us/step - loss: 0.9937 - acc: 0.5868 - val_loss: 0.9901 - val_acc: 0.5879
Epoch 25/80
8120/8120 [==============================] - 8s 964us/step - loss: 0.9936 - acc: 0.5868 - val_loss: 0.9889 - val_acc: 0.5879
Epoch 26/80
8120/8120 [==================

Epoch 18/80
8121/8121 [==============================] - 8s 955us/step - loss: 0.8218 - acc: 0.6759 - val_loss: 0.7782 - val_acc: 0.6892
Epoch 19/80
8121/8121 [==============================] - 8s 963us/step - loss: 0.8317 - acc: 0.6670 - val_loss: 0.7714 - val_acc: 0.6759
Epoch 20/80
8121/8121 [==============================] - 8s 983us/step - loss: 0.8173 - acc: 0.6677 - val_loss: 0.7341 - val_acc: 0.6946
Epoch 21/80
8121/8121 [==============================] - 8s 990us/step - loss: 0.8222 - acc: 0.6697 - val_loss: 0.7377 - val_acc: 0.7064
Epoch 22/80
8121/8121 [==============================] - 8s 962us/step - loss: 0.7968 - acc: 0.6817 - val_loss: 0.7719 - val_acc: 0.7049
Epoch 23/80
8121/8121 [==============================] - 8s 946us/step - loss: 0.7930 - acc: 0.6877 - val_loss: 0.7278 - val_acc: 0.7034
Epoch 24/80
8121/8121 [==============================] - 8s 960us/step - loss: 0.8311 - acc: 0.6643 - val_loss: 0.8033 - val_acc: 0.6877
Epoch 25/80
8121/8121 [==================

Epoch 10/80
8121/8121 [==============================] - 8s 968us/step - loss: 6.7084 - acc: 0.5838 - val_loss: 6.4472 - val_acc: 0.6000
Epoch 11/80
8121/8121 [==============================] - 8s 952us/step - loss: 6.7084 - acc: 0.5838 - val_loss: 6.4472 - val_acc: 0.6000
Epoch 12/80
8121/8121 [==============================] - 8s 973us/step - loss: 6.7084 - acc: 0.5838 - val_loss: 6.4472 - val_acc: 0.6000
Epoch 13/80
8121/8121 [==============================] - 8s 955us/step - loss: 6.7084 - acc: 0.5838 - val_loss: 6.4472 - val_acc: 0.6000
Epoch 14/80
8121/8121 [==============================] - 8s 957us/step - loss: 6.7084 - acc: 0.5838 - val_loss: 6.4472 - val_acc: 0.6000
Epoch 15/80
8121/8121 [==============================] - 8s 970us/step - loss: 6.7084 - acc: 0.5838 - val_loss: 6.4472 - val_acc: 0.6000
Epoch 16/80
8121/8121 [==============================] - 8s 964us/step - loss: 6.7084 - acc: 0.5838 - val_loss: 6.4472 - val_acc: 0.6000
Epoch 17/80
8121/8121 [==================

In [ ]:
for bat, lr in itertools.product(batch_S, learning_rate):
    print(bat, lr)

In [5]:
import pprint
pprint.pprint(F1_result)

[array([0.77229626, 0.64092216, 0.90089052, 0.74408891]), array([0.784631  , 0.62502861, 0.90086941, 0.74591783]), array([0.78980568, 0.58529064, 0.90681964, 0.75824655]), array([0.79716358, 0.55634864, 0.90042693, 0.73499462]), array([0.       , 0.       , 0.7397273, 0.       ]), array([0.77738419, 0.6487298 , 0.89984665, 0.72402845]), array([0.7682173 , 0.65503487, 0.90150719, 0.74518419]), array([0.78919052, 0.61752665, 0.90305826, 0.74829142]), array([0.7983177 , 0.6010097 , 0.90163881, 0.74496582]), array([0.       , 0.       , 0.7397273, 0.       ]), array([0.73000107, 0.62021797, 0.89655591, 0.706681  ]), array([0.75950593, 0.65094019, 0.90276777, 0.73426779]), array([0.77125078, 0.64762685, 0.89839041, 0.74110783]), array([0.79250755, 0.62614335, 0.90266635, 0.74891516]), array([0.       , 0.       , 0.7397273, 0.       ]), array([0.703972  , 0.62204042, 0.89622738, 0.67817894]), array([0.76460019, 0.62968524, 0.90051409, 0.74686673]), array([0.79047648, 0.61877243, 0.90490228,

In [ ]:
array([0.79250755, 0.62614335, 0.90266635, 0.74891516])